# Stock Selection for the Defensive Investor
**Purpose.** Purpose of this notebook is to select companies that pass Ben Graham's test criteria for the *defensive investor*, as described in *Chapter 14 of Intelligent Investor*.
The criteria has 7 tests:
1. **Adequate Size of the Enterprise.** Not less than \\$100 million of annual sales for an industrial company and, not less than \\$50 million of total assets for a public utility.


2. **A Sufficiently Strong Financial Condition.** For industrial companies current assets should be at least twice current liabilities—a so-called two-to-one current ratio. Also, long-term debt should not exceed the net current assets (or “working capital”). For public utilities the debt should not exceed twice the stock equity (at book value).


3. **Earnings Stability.** Some earnings for the common stock in each of the past ten years.


4. **Dividend Record.** Uninterrupted payments for at least the past 20 years.


5. **Earnings Growth.** A minimum increase of at least one-third in per-share earnings in the past ten years using three-year averages at the beginning and end.


6. **Moderate Price/Earnings Ratio.** Current price should not be more than 15 times average earnings of the past three years.


7. **Moderate Ratio of Price to Assets.** Current price should not be more than 1½ times the book value last reported. However, a multiplier of earnings below 15 could justify a correspondingly higher multiplier of assets. As a rule of thumb we suggest that the product of the multiplier times the ratio of price to book value should not exceed 22.5.

In [1]:
import sys
import pandas as pd
import numpy as np
import quandl
import os
import warnings
from datetime import datetime
from tqdm import tqdm_notebook as tqdm
#import humanize

warnings.filterwarnings('ignore')
quandl.ApiConfig.api_key = os.getenv('QUANDL_APY_KEY')
print(f'Python version: {sys.version}')
print(f'Pandas version: {pd.__version__}\nNumpy version: {np.__version__}')

Python version: 3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Pandas version: 0.23.4
Numpy version: 1.15.4


## Getting necessary data for a given company
First step is to get necessary data to determine if a given company pass or fails Ben Graham's test criteria for the *defensive investor*, as described in *Chapter 14 of Intelligent Investor*. Using [Quandl SF1 | Core US Fundamentals Data](https://www.quandl.com/databases/SF1)

In [2]:
ticker = 'AAPL'

def get_data(ticker, dt=datetime.today()): 
    columns = ['calendardate', 'datekey', 'revenue', 'currentratio', 'eps', 'dps', 'bvps', 'price']
    df = quandl.get_table('SHARADAR/SF1', dimension='ARY', ticker=ticker, qopts={"columns":columns})
    df = df[df['datekey'] <= dt]
    df.sort_values('calendardate', inplace=True)
    return df

get_data(ticker).tail()

,calendardate,datekey,revenue,currentratio,eps,dps,bvps,price
None,,,,,,,,
4,2014-12-31,2014-10-27,182795000000,1.080,6.49,1.811,18.798,105.11
3,2015-12-31,2015-10-28,233715000000,1.109,9.28,1.980,21.136,119.27
2,2016-12-31,2016-10-26,215639000000,1.353,8.35,2.180,23.896,115.59
1,2017-12-31,2017-11-03,229234000000,1.276,9.27,2.400,26.031,172.50
0,2018-12-31,2018-11-05,265595000000,1.124,12.01,2.720,22.315,207.48


In [3]:
get_data(ticker, dt=datetime(2018, 6, 1)).tail()

,calendardate,datekey,revenue,currentratio,eps,dps,bvps,price
None,,,,,,,,
5,2013-12-31,2013-10-30,170910000000,1.679,5.719,1.629,19.521,74.985
4,2014-12-31,2014-10-27,182795000000,1.080,6.490,1.811,18.798,105.110
3,2015-12-31,2015-10-28,233715000000,1.109,9.280,1.980,21.136,119.270
2,2016-12-31,2016-10-26,215639000000,1.353,8.350,2.180,23.896,115.590
1,2017-12-31,2017-11-03,229234000000,1.276,9.270,2.400,26.031,172.500


## Check if a given company pass the 7 criteria
Second step is to implement the criteria and check if a given company pass it or not.

In [19]:
def criteria_defensive_investor(ticker_or_list, verbose=False, dt=datetime.today()):
    list_tickers = []
    if isinstance(ticker_or_list, str):
        list_tickers.append(ticker_or_list)
    else:
        list_tickers = ticker_or_list
        
    summary_columns = ['ticker', 'last_date', 'first_date', 'revenue', 'current_ratio', 
                       'positive_eps_p10yrs_count', 'dividend_distribution_p20yrs_count', 
                       'earnings_change_p10yrs', 'pe', 'pb', 'pexpb', 'size_criteria', 
                       'financial_condition_criteria', 'earnings_stability_criteria', 
                       'dividend_record_criteria', 'earnings_growth_criteria', 'pe_criteria', 
                       'pb_criteria', 'full_criteria']
    df_ = pd.DataFrame(columns=summary_columns)
        
    for ticker in tqdm(list_tickers):
        if verbose:
            print('\nTest for ' + ticker + ':')
        data = get_data(ticker, dt=dt)

        # In case of no data
        if data.shape[0] == 0:  # No data
            if verbose:
                print('- No data available')
            df_.append(pd.Series([float('NaN')] * len(summary_columns), index=summary_columns), ignore_index=True)
            df_.at[df_.index[-1], 'ticker'] = ticker
            continue

        # Size criteria
        size_criteria = data['revenue'].values[-1] > 100000000

        # Financial condition criteria
        if data['currentratio'].values[-1] is not None:
            current_ratio = data['currentratio'].values[-1]
            financial_condition_criteria = current_ratio > 2
        else:
            current_ratio = float('NaN')
            financial_condition_criteria = False

        # Earnings stability criteria
        earnings_stability_criteria = (data['eps'].tail(10) > 0).all()

        # Dividends record criteria
        dividend_record_criteria = (data['dps'].tail(20) > 0).all()

        # Earnings growth criteria
        last_year = pd.to_datetime(data['calendardate'].values[-1]).year
        eps_0 = data[(data['calendardate'].dt.year > last_year - 13) & 
                     (data['calendardate'].dt.year <= last_year - 10)]['eps'].mean()
        eps_1 = data[(data['calendardate'].dt.year > last_year - 3) & 
                     (data['calendardate'].dt.year <= last_year)]['eps'].mean()
        earnings_growth_criteria = (np.float64(eps_1) / eps_0) > 1.33

        # P/E ratio criteria
        current_price = data['price'].values[-1]
        pe = current_price / eps_1
        pe_criteria = pe < 15

        # Price to Assets criteria
        pb = current_price / data['bvps'].values[-1]
        pb_criteria = pb < 1.5
        if (pe * pb < 22.5):
            pb_criteria = True

        # Full criteria
        full_criteria = size_criteria and financial_condition_criteria and earnings_stability_criteria \
            and dividend_record_criteria and earnings_growth_criteria and pe_criteria and pb_criteria
        
        # Add to dataframe
        my_dic = {'ticker': ticker, 
                  'last_date': data['calendardate'].values[-1], 
                  'first_date': data['calendardate'].values[0], 
                  'revenue': data['revenue'].values[-1], 
                  'current_ratio': current_ratio, 
                  'positive_eps_p10yrs_count': data.tail(10)[data['eps'] > 0]['eps'].count(), 
                  'dividend_distribution_p20yrs_count': data.tail(20)[data['dps'] > 0]['dps'].count(), 
                  'earnings_change_p10yrs': (np.float64(eps_1) / eps_0), 
                  'pe': pe, 
                  'pb': pb, 
                  'pexpb': pe * pb,
                  'size_criteria': size_criteria, 
                  'financial_condition_criteria': financial_condition_criteria, 
                  'earnings_stability_criteria': earnings_stability_criteria, 
                  'dividend_record_criteria': dividend_record_criteria, 
                  'earnings_growth_criteria': earnings_growth_criteria, 
                  'pe_criteria': pe_criteria, 
                  'pb_criteria': pb_criteria, 
                  'full_criteria': full_criteria}
        df_.loc[len(df_)] = my_dic 

        if verbose:
            print('- Size criteria: \t\t' + str(size_criteria) + 
                  '\tRevenues of $' + humanize.intword(data['revenue'].values[-1]) + ' (threshold is $100 million)')
            print('- Financial condition criteria: ' + str(financial_condition_criteria) + 
                  '\tCurrent ratio of %1.2f' % current_ratio + ' (threshold is 2.0)')
            print('- Earnings stability criteria: \t' + str(earnings_stability_criteria) + 
                  '\tPositive earnings in %d of past 10 years' % data.tail(10)[data['eps'] > 0]['eps'].count())
            print('- Dividend record criteria: \t' + str(dividend_record_criteria) + 
                  '\tDistribution of dividend in %d of past 20 years' % data.tail(20)[data['dps'] > 0]['dps'].count())
            print('- Earnings growth criteria: \t' + str(earnings_growth_criteria) + 
                  '\tEarnings change of %+.0f%%' % (100 * ((np.float64(eps_1) / eps_0) - 1)) + ' in past 10 years (minimum is +33%)')
            print('- Moderate P/E ratio criteria: \t' + str(pe_criteria) + 
                  '\tCurrent price is %1.1fx avg P3yrs earnings (limit is 15)' % pe)
            print('- Moderate P/B ratio criteria: \t' + str(pb_criteria) + 
                  '\tCurrent price is %1.1fx last book value (limit 1.5), \n\t\t\t\t\tand PE * PB is %1.1f (limit 22.5)' % (pb, pe * pb))
            print('- Full criteria: \t\t' + str(full_criteria))
    
    return df_


In [20]:
criteria_defensive_investor('AAPL').transpose()

,0
ticker,AAPL
last_date,2018-12-31 00:00:00
first_date,1997-12-31 00:00:00
revenue,265595000000
current_ratio,1.124
positive_eps_p10yrs_count,10
dividend_distribution_p20yrs_count,7
earnings_change_p10yrs,17.4602
pe,21.0071
pb,9.29778


In [21]:
criteria_defensive_investor('AAPL', dt=datetime(2018, 6, 1)).transpose()

,0
ticker,AAPL
last_date,2017-12-31 00:00:00
first_date,1997-12-31 00:00:00
revenue,229234000000
current_ratio,1.276
positive_eps_p10yrs_count,10
dividend_distribution_p20yrs_count,6
earnings_change_p10yrs,23.3913
pe,19.2379
pb,6.62671


## Bulk test it in DJIA companies

In [22]:
def get_djia_symbols():
    data = pd.read_html("https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average")
    table = data[1]
    sliced_table = table[1:]
    header = table.iloc[0]
    corrected_table = sliced_table.rename(columns=header)
    return corrected_table['Symbol'].tolist()

In [23]:
df_djia = criteria_defensive_investor(get_djia_symbols(), dt=datetime(2009, 1, 1))
df_djia.head()

,ticker,last_date,first_date,revenue,current_ratio,positive_eps_p10yrs_count,dividend_distribution_p20yrs_count,earnings_change_p10yrs,pe,pb,pexpb,size_criteria,financial_condition_criteria,earnings_stability_criteria,dividend_record_criteria,earnings_growth_criteria,pe_criteria,pb_criteria,full_criteria
0,MMM,2007-12-31,1995-12-31,24462000000,1.835,10,13,2.705671,15.958084,4.839881,77.235232,True,False,True,True,True,False,False,False
1,AXP,2007-12-31,1993-12-31,27731000000,NaN,10,15,2.524555,14.012618,4.724527,66.202989,True,False,True,True,True,True,False,False
2,AAPL,2008-12-31,1997-12-31,32479000000,2.462,9,0,-5.336478,26.087802,4.356953,113.663329,True,True,False,False,False,False,False,False
3,BA,2007-12-31,1997-12-31,66387000000,0.865,10,11,-21.370370,22.143847,7.133406,157.961053,True,False,True,True,False,False,False,False
4,CAT,2007-12-31,1997-12-31,44958000000,1.145,10,11,2.271772,14.113681,5.113873,72.175576,True,False,True,True,True,True,False,False


### Analysis
Let's analyze how many companies pass stock selection criteria for the defensive investor as proposed by Ben Graham, today.

In [24]:
def analysis(df):
    c1 = len(df[df['size_criteria'] == True]['ticker'].tolist())
    c2 = len(df[df['financial_condition_criteria'] == True]['ticker'].tolist())
    c3 = len(df[df['earnings_stability_criteria'] == True]['ticker'].tolist())
    c4 = len(df[df['dividend_record_criteria'] == True]['ticker'].tolist())
    c5 = len(df[df['earnings_growth_criteria'] == True]['ticker'].tolist())
    c6 = len(df[df['pe_criteria'] == True]['ticker'].tolist())
    c7 = len(df[df['pb_criteria'] == True]['ticker'].tolist())

    c8 = len(df[(df['size_criteria'] == True) & 
                (df['financial_condition_criteria'] == True) & 
                (df['earnings_stability_criteria'] == True) & 
                (df['dividend_record_criteria'] == True) & 
                (df['earnings_growth_criteria'] == True)]['ticker'].tolist())

    cdi = len(df[df['full_criteria'] == True]['ticker'].tolist())

    print(f'{c1:4d} companies passing size criteria of minimum $100 million revenues')
    print(f'{c2:4d} companies passing financial condition criteria of minimum 2 current ratio')
    print(f'{c3:4d} companies passing earnings stability criteria of positive earnings in past 10 years')
    print(f'{c4:4d} companies passing dividend record criteria of uninterrupted payments in past 20 years')
    print(f'{c5:4d} companies passing earnings growth criteria of minimum 33% growth in past 10 years')
    print(f'{c6:4d} companies passing moderate PE ratio criteria of maximum 15')
    print(f'{c7:4d} companies passing moderate PB ratio criteria of maximum 1.5, or PE * PB of maximum 22.5')
    print(f'{c8:4d} companies passing all except moderate PE and PB ratio criteria')
    print(f'{cdi:4d} companies passing all criteria')
    
    df['count_successful_criteria'] = df.apply(lambda row: sum([row['size_criteria'], 
                                                                row['financial_condition_criteria'], 
                                                                row['earnings_stability_criteria'], 
                                                                row['dividend_record_criteria'], 
                                                                row['earnings_growth_criteria'], 
                                                                row['pe_criteria'], 
                                                                row['pb_criteria']]), axis=1)


In [25]:
analysis(df_djia)

print('\nCompanies passing first 5 criteria: ' + 
      str(df_djia[(df_djia['size_criteria'] == True) & 
              (df_djia['financial_condition_criteria'] == True) & 
              (df_djia['earnings_stability_criteria'] == True) & 
              (df_djia['dividend_record_criteria'] == True) & 
              (df_djia['earnings_growth_criteria'] == True)]['ticker'].tolist()))

print('Companies passing all criteria: '+ str(df_djia[(df_djia['full_criteria'] == True)]['ticker'].tolist()))

  30 companies passing size criteria of minimum $100 million revenues
   5 companies passing financial condition criteria of minimum 2 current ratio
  24 companies passing earnings stability criteria of positive earnings in past 10 years
  24 companies passing dividend record criteria of uninterrupted payments in past 20 years
  21 companies passing earnings growth criteria of minimum 33% growth in past 10 years
  12 companies passing moderate PE ratio criteria of maximum 15
   5 companies passing moderate PB ratio criteria of maximum 1.5, or PE * PB of maximum 22.5
   2 companies passing all except moderate PE and PB ratio criteria
   0 companies passing all criteria

Companies passing first 5 criteria: ['NKE', 'PFE']
Companies passing all criteria: []


As we can see, no company pass all 7 criteria. **Nike** is the closest one, passing all first 5, but failing the last 2 criteria, which are related to company's current price attractiveness.

Now, let's count how many criteria each company pass and analyze that.

In [27]:
df_djia.sort_values('count_successful_criteria', ascending=False) #[['ticker', 'count_successful_criteria']]

,ticker,last_date,first_date,revenue,current_ratio,positive_eps_p10yrs_count,dividend_distribution_p20yrs_count,earnings_change_p10yrs,pe,pb,pexpb,size_criteria,financial_condition_criteria,earnings_stability_criteria,dividend_record_criteria,earnings_growth_criteria,pe_criteria,pb_criteria,full_criteria,count_successful_criteria
15,JPM,2007-12-31,1997-12-31,71372000000,NaN,10,11,1.337188,10.986486,1.120514,12.310510,True,False,True,True,True,True,True,False,6
20,PFE,2007-12-31,1997-12-31,48418000000,2.146,10,11,2.810903,13.503030,2.322285,31.357882,True,True,True,True,True,True,False,False,6
28,WBA,2008-12-31,1998-12-31,59034000000,1.570,10,12,3.728155,13.125000,1.938163,25.438394,True,False,True,True,True,True,False,False,5
4,CAT,2007-12-31,1997-12-31,44958000000,1.145,10,11,2.271772,14.113681,5.113873,72.175576,True,False,True,True,True,True,False,False,5
5,CVX,2007-12-31,1997-12-31,220904000000,1.165,10,12,3.227364,11.099751,2.398103,26.618351,True,False,True,True,True,True,False,False,5
8,DWDP,2007-12-31,1997-12-31,53513000000,1.499,8,11,1.484185,10.141501,1.952459,19.800868,True,False,False,True,True,True,True,False,5
9,XOM,2007-12-31,1997-12-31,404552000000,1.474,10,11,3.870968,13.542424,3.927238,53.184317,True,False,True,True,True,True,False,False,5
10,GS,2007-12-31,1999-12-31,45987000000,NaN,10,10,NaN,9.968644,1.939438,19.333564,True,False,True,True,False,True,True,False,5
11,HD,2007-12-31,1996-12-31,77349000000,1.155,10,12,5.487340,11.082174,2.748049,30.454353,True,False,True,True,True,True,False,False,5
19,NKE,2008-12-31,1998-12-31,18627000000,2.661,10,11,4.575581,18.182973,3.587365,65.228967,True,True,True,True,True,False,False,False,5


In [28]:
# TODO: Plot top companies performance & compare with SPY

## Bulk test it in S&P500 companies

In [29]:
def get_sp500_symbols():
    data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    table = data[1]
    sliced_table = table[1:]
    header = table.iloc[0]
    corrected_table = sliced_table.rename(columns=header)
    return corrected_table['Symbol'].tolist()

In [30]:
df_sp500 = criteria_defensive_investor(get_sp500_symbols(), dt=datetime(2009, 1, 1))
df_sp500.head()

,ticker,last_date,first_date,revenue,current_ratio,positive_eps_p10yrs_count,dividend_distribution_p20yrs_count,earnings_change_p10yrs,pe,pb,pexpb,size_criteria,financial_condition_criteria,earnings_stability_criteria,dividend_record_criteria,earnings_growth_criteria,pe_criteria,pb_criteria,full_criteria
0,MMM,2007-12-31,1995-12-31,24462000000,1.835,10,13,2.705671,15.958084,4.839881,77.235232,True,False,True,True,True,False,False,False
1,ABBV,2007-12-31,1997-12-31,25914238000,1.543,10,11,1.379902,29.611012,4.833014,143.110451,True,False,True,True,True,False,False,False
2,ABMD,2008-12-31,1998-12-31,58941000,3.671,0,0,7.397849,-17.267442,6.898955,-119.127299,False,True,False,False,True,True,True,False
3,ACN,2008-12-31,2001-12-31,25313826000,1.338,7,4,NaN,14.750000,7.738645,114.145009,True,False,False,False,False,True,False,False
4,ATVI,2008-12-31,1998-12-31,2898136000,3.560,9,0,10.576923,61.363636,5.085895,312.089019,True,True,False,False,True,False,False,False


In [32]:
analysis(df_sp500)

print('\nCompanies passing first 5 criteria: ' + 
      str(df_sp500[(df_sp500['size_criteria'] == True) & 
              (df_sp500['financial_condition_criteria'] == True) & 
              (df_sp500['earnings_stability_criteria'] == True) & 
              (df_sp500['dividend_record_criteria'] == True) & 
              (df_sp500['earnings_growth_criteria'] == True)]['ticker'].tolist()))

print('Companies passing all criteria: '+ str(df_sp500[(df_sp500['full_criteria'] == True)]['ticker'].tolist()))

 442 companies passing size criteria of minimum $100 million revenues
 107 companies passing financial condition criteria of minimum 2 current ratio
 266 companies passing earnings stability criteria of positive earnings in past 10 years
 253 companies passing dividend record criteria of uninterrupted payments in past 20 years
 229 companies passing earnings growth criteria of minimum 33% growth in past 10 years
 161 companies passing moderate PE ratio criteria of maximum 15
 129 companies passing moderate PB ratio criteria of maximum 1.5, or PE * PB of maximum 22.5
  17 companies passing all except moderate PE and PB ratio criteria
   2 companies passing all criteria

Companies passing first 5 criteria: ['ALB', 'BRK-B', 'CTAS', 'XRAY', 'BEN', 'GWW', 'HRS', 'HP', 'ITW', 'LB', 'MDT', 'NKE', 'NUE', 'PFE', 'SYK', 'TWTR', 'TSS']
Companies passing all criteria: ['BEN', 'HP']


As we can see, still no company pass all 7 criteria. But now, we can see that **17 companies** passed all first 5, but failed the last 2 criteria, which are related to company's current price attractiveness.

Now, let's count how many criteria each company pass and analyze that.

In [33]:
#df_sp500.sort_values('count_successful_criteria', ascending=False)[['ticker', 'count_successful_criteria']]
df_sp500[df_sp500['count_successful_criteria'] == 6]

,ticker,last_date,first_date,revenue,current_ratio,positive_eps_p10yrs_count,dividend_distribution_p20yrs_count,earnings_change_p10yrs,pe,pb,pexpb,size_criteria,financial_condition_criteria,earnings_stability_criteria,dividend_record_criteria,earnings_growth_criteria,pe_criteria,pb_criteria,full_criteria,count_successful_criteria
12,APD,2008-12-31,1998-12-31,10414500000,1.288,10,11,1.619423,11.304700,1.944793,21.985301,True,False,True,True,True,True,True,False,6
22,ALL,2007-12-31,1997-12-31,36769000000,NaN,10,11,1.714685,8.079935,1.287731,10.404783,True,False,True,True,True,True,True,False,6
30,AIG,2007-12-31,1997-12-31,110064000000,NaN,10,11,2.332879,12.728426,1.335923,17.004192,True,False,True,True,True,True,True,False,6
39,ADI,2008-12-31,1997-12-31,2582931000,3.672,10,6,4.314607,8.828125,2.037015,17.983021,True,True,True,False,True,True,True,False,6
59,BAC,2007-12-31,1997-12-31,66319000000,NaN,10,11,1.890710,10.260941,1.247515,12.800680,True,False,True,True,True,True,True,False,6
62,BBT,2007-12-31,1997-12-31,6206000000,NaN,10,11,2.271698,10.877076,1.419836,15.443665,True,False,True,True,True,True,True,False,6
79,CPRI,2007-12-31,1997-12-31,11947566000,NaN,10,13,6.548241,7.345213,0.706697,5.190840,True,False,True,True,True,True,True,False,6
90,CTL,2007-12-31,1997-12-31,2656241000,0.397,10,11,2.519810,11.380503,1.157154,13.168998,True,False,True,True,True,True,True,False,6
100,CINF,2007-12-31,1997-12-31,4259000000,NaN,10,11,2.543278,8.074584,1.037341,8.376096,True,False,True,True,True,True,True,False,6
101,CTAS,2008-12-31,1998-12-31,3937900000,3.492,10,11,2.556795,13.948304,1.962228,27.369754,True,True,True,True,True,True,False,False,6


Here we can see 2 interesting things. From all S&P 500 companies, **8 companies above passed all but 1 criteria** that Ben Graham proposed:
- 5 companies have missing current ratios. We already contacted Quandl team to fix that.
- 3 companies have all data. From those:
    - **NWL (Newell Brands Inc)** got pretty close. It failed on earnings stability criteria, getting positive EPS in 9 out of past 10 years.
    - **LB (Newell Brands Inc)** also got pretty close. It failed on financial condition criteria, getting 1.6 current ratio, below minimum threshold of 2.
    - **TAP (Molson Coors Brewing Company)** failed on financial condition criteria as well, but got far from minimum current ratio threshold with 0.6.

## Next steps:
1. Parametrize `criteria_defensive_investor` by date, adjusting it to return companies passing criteria in a given past date **DONE!**
2. Ask for Quandl help to fix companies without current ratio (implement fallback if not fixed)

S&P500 companies without current ratio in Quandl Sharadar Core US Fundamentals Data:

In [122]:
df_sp500[df_sp500['current_ratio'].isnull()]['ticker'].tolist()

['AMG',
 'AFL',
 'ARE',
 'ALL',
 'AXP',
 'AIG',
 'AMP',
 'AIV',
 'AIZ',
 'AVB',
 'BAC',
 'BK',
 'BBT',
 'BLK',
 'BXP',
 'BHF',
 'COF',
 'SCHW',
 'CB',
 'CI',
 'CINF',
 'C',
 'CFG',
 'CMA',
 'DHI',
 'DE',
 'DLR',
 'DFS',
 'DRE',
 'ETFC',
 'EQR',
 'ESS',
 'RE',
 'EXR',
 'FRT',
 'FITB',
 'BEN',
 'GE',
 'GS',
 'HIG',
 'HCP',
 'HST',
 'HBAN',
 'IVZ',
 'JEF',
 'JPM',
 'KEY',
 'KIM',
 'LEN',
 'LIN',
 'L',
 'MTB',
 'MAC',
 'MET',
 'MAA',
 'MS',
 'NTRS',
 'PBCT',
 'PNC',
 'PFG',
 'PGR',
 'PLD',
 'PRU',
 'PSA',
 'PHM',
 'RJF',
 'O',
 'REG',
 'RF',
 'SPG',
 'SLG',
 'STT',
 'STI',
 'SIVB',
 'SYF',
 'TROW',
 'TMK',
 'TRV',
 'UDR',
 'USB',
 'UNM',
 'VTR',
 'VNO',
 'WFC',
 'WELL',
 'WU',
 'ZION']